# [Projet 6: Catégorisez automatiquement des questions](https://openclassrooms.com/fr/projects/categorisez-automatiquement-des-questions)
(parcours data: [here](https://openclassrooms.com/paths/63-data-scientist))

## Skeleton for API

### Imports

In [1]:
import os
HOME = os.path.expanduser('~/')
HOST = os.uname()[1]
if HOST == 'Arthurs-MacBook-Pro.local':
    os.chdir(HOME+'Documents/GitHub/OCDataSciencePath/Project6/Work/')    # @home
else:
    raise ValueError('unknown host: {}'.format(HOST))
    
import numpy as np
import pandas as pd
import pickle

In [2]:
from Pkg.helper import basicHTMLTextCleaner # needs correct working directory

### Data

In [3]:
if HOST == 'Arthurs-MacBook-Pro.local':
    pathToDataDir = HOME+'Documents/Dropbox/Transit/OCDataScienceData/Project6/Data/'    # @home
    pathToIntellDir = HOME+'Documents/Dropbox/Transit/OCDataScienceData/Project6/Intelligence/'    # @home
else:
    raise ValueError('unknown host: {}'.format(HOST))

In [4]:
tokenizer = pickle.load(open(pathToIntellDir+'tokenizer.pkl','rb'))
stopwords = pickle.load(open(pathToIntellDir+'stopwords.pkl','rb'))
stemer = pickle.load(open(pathToIntellDir+'stemer.pkl','rb'))

count_title = pickle.load(open(pathToIntellDir+'count_title.pkl','rb'))
count_body = pickle.load(open(pathToIntellDir+'count_body.pkl','rb'))
tfidf_title = pickle.load(open(pathToIntellDir+'tfidf_title.pkl','rb'))
tfidf_body = pickle.load(open(pathToIntellDir+'tfidf_body.pkl','rb'))

model = pickle.load(open(pathToIntellDir+'model.pkl','rb'))
count_tag = pickle.load(open(pathToIntellDir+'count_tag.pkl','rb'))

## API call

### Inputs

In [5]:
check = True

In [6]:
if check:
    filename = 'QueryResults_10k.csv'
    df = pd.read_csv(pathToDataDir+filename,index_col='Id')

    i = 35
    title = df.iloc[i]['Title']
    body = df.iloc[i]['Body']
    tag = df.iloc[i]['Tags']    
    
else:    
    title = 'How to shut down SQL?'
    body = 'I don\'t know how to set my computer off. Please help me.'

### Data processing

In [7]:
# clean text
title_c = basicHTMLTextCleaner(title,tokenizer,stopwords,stemer)
body_c = basicHTMLTextCleaner(body,tokenizer,stopwords,stemer)
# count frequencies
title_f = count_title.transform([title_c])
body_f = count_body.transform([body_c])
# get TF-IDF
title_i = tfidf_title.transform(title_f)
body_i = tfidf_body.transform(body_f)
# get X
x = np.hstack((body_i.toarray(),title_i.toarray())) # SEE HOW TO AVOID .toarray()...

In [8]:
if 0 == title_i.sum():
    print('WARN - No keyword found in the title: please be more precise.')
if 0 == body_i.sum():
    print('WARN - No keyword found in the body: please be more precise.')

### Prediction

In [9]:
y_hat = model.predict(x)
tag_hat = count_tag.inverse_transform(y_hat)
tag_hat = list(tag_hat[0])

In [10]:
if 0 == len(tag_hat):
    print('WARN - Did not predict any tag: maybe the question deals with rare tags?')

In [11]:
print('prediction:\n{}\n'.format(tag_hat))
if check: print('true:\n{}'.format(tag))

prediction:
['<asp.net>']

true:
<.net><asp.net><svn><versioning>
